### Import Packages

In [153]:
import pandas as pd
import numpy as np 
import pybedtools
# import argparse

# Work Space for priorCDR.py

In [154]:
# would be from argparse
'''
    parser = argparse.ArgumentParser(description='Process bedMethyl and regions BED files.')
    parser.add_argument('bedMethyl_path', type=str, help='Path to the bedMethyl file')
    parser.add_argument('cenSat_path', type=str, help='Path to the regions BED file')
    parser.add_argument('mod_code', type=str, help='Modification code to filter bedMethyl file')
    parser.add_argument('sat_type', type=str, help='Satellite type to filter regions file')
    args = parser.parse_args()
'''
bedMethyl_path = 'chr10_MAT_HG002_ONT.5mCpileup.bed'

cenSat_path = 'chr10_MAT_hg002v1.0.1.cenSatv2.0.bed'

mod_code = 'm'

sat_type = 'H1L'

window_size = 1020 # 6 repeat units?

priorCDR_percent = 5

priorTransition_percent = 10

minCDR_size = 3000

depletion = True

output_label = "CDR"

In [155]:
def filter_bedMethyl(bedMethyl_path, mod_code):
    # Read the bedMethyl file, keeping only the necessary columns
    df = pd.read_csv(bedMethyl_path, sep='\t', header=None, usecols=[0, 1, 2, 3, 10])
    # Filter rows based on the mod_code
    filtered_df = df[df[3] == mod_code]
    # Drop the third column (index 2)
    filtered_df = filtered_df.drop(columns=[3])
    return filtered_df

In [156]:
def filter_regions(regions_path, sat_type):
    # Read the regions file
    regions_df = pd.read_csv(regions_path, sep='\t', header=None)
    # Filter rows based on the sat_type substring in column 4
    filtered_regions_df = regions_df[regions_df[3].str.contains(sat_type)]
    return filtered_regions_df

In [157]:
def intersect_files(filtered_bedMethyl, filtered_regions):
    # Create BedTool objects
    bedMethyl_bedtool = pybedtools.BedTool.from_dataframe(filtered_bedMethyl)
    regions_bedtool = pybedtools.BedTool.from_dataframe(filtered_regions)
    # Intersect the files, keeping only the bedMethyl values
    intersected = bedMethyl_bedtool.intersect(regions_bedtool, wa=True, u=True)
    # Convert to DataFrame
    intersected_df = intersected.to_dataframe(names=[0, 1, 2, 3, 10])
    return intersected_df

In [158]:
def create_regions(intersected_df, window_size=1020):
    # Compute the minimum and maximum values in column 1 (the start positions)
    min_val = int( intersected_df[1].min() )
    max_val = int( intersected_df[1].max() )
    
    # Create a list to hold the region data
    regions = []
    
    # Generate 1020bp regions from min_val to max_val
    current_start = min_val
    while current_start + window_size <= max_val:
        current_end = current_start + window_size
        regions.append([intersected_df[0][0], current_start, current_end])
        current_start = current_end

    # Convert the list to a DataFrame
    regions_df = pd.DataFrame(regions, columns=[0, 1, 2])
    
    return regions_df

In [166]:
def calculate_mean_within_windows(intersected_df, regions_df):
    # Create BedTool objects
    intersected_bedtool = pybedtools.BedTool.from_dataframe(intersected_df)
    regions_bedtool = pybedtools.BedTool.from_dataframe(regions_df)
    
    # Use BedTool map to calculate the mean of column 10 within each window
    result = regions_bedtool.map(intersected_bedtool, c=4, o='mean')
    
    # Convert the result to a DataFrame
    result_df = result.to_dataframe(names=[0, 1, 2, 'mean_value'])
    
    return result_df

In [160]:
def calculate_percentiles(windows_mean_df, priorCDR_percentile=5, priorTransition_percentile=10):
    # Replace '.' with NaN
    windows_mean_df['mean_value'].replace('.', np.nan, inplace=True)
    # Drop NaN values
    mean_values = windows_mean_df['mean_value'].dropna().astype(float)
    # Calculate prior and transition scores as certain percentiles
    priorCDR_score = np.percentile(mean_values, q=priorCDR_percentile)
    priorTransition_score = np.percentile(mean_values, q=priorTransition_percentile)
    
    return priorCDR_score, priorTransition_score

In [161]:
def create_priorCDR_dataframe(windows_mean_df, priorCDR_score, minCDR_size):
    # Replace '.' with NaN and convert to float
    windows_mean_df['mean_value'] = pd.to_numeric(windows_mean_df['mean_value'], errors='coerce')
    # Drop NaN values
    windows_mean_df = windows_mean_df.dropna(subset=['mean_value'])
    # Filter windows with mean_value below priorCDR_score
    windows_below_priorCDR_score = windows_mean_df[windows_mean_df['mean_value'] < priorCDR_score]
    # Create a BedTool object from the filtered windows
    windows_bedtool = pybedtools.BedTool.from_dataframe(windows_below_priorCDR_score)
    # Merge adjacent windows
    merged_windows = windows_bedtool.merge()
    # Convert the merged result to a DataFrame
    merged_df = merged_windows.to_dataframe(names=[0, 1, 2])
    # Calculate the size of each merged region
    merged_df['size'] = merged_df[2] - merged_df[1]
    # Filter merged regions by size threshold
    filtered_merged_df = merged_df[merged_df['size'] >= minCDR_size]
    # Drop the 'size' column before returning
    filtered_merged_df = filtered_merged_df.drop(columns=['size'])
    return filtered_merged_df

In [162]:
def create_priorTransition_dataframe(windows_mean_df, priorTransition_score, CDR_df):
    # Replace '.' with NaN and convert to float
    windows_mean_df['mean_value'] = pd.to_numeric(windows_mean_df['mean_value'], errors='coerce')
    # Drop NaN values
    windows_mean_df = windows_mean_df.dropna(subset=['mean_value'])
    # Filter windows with mean_value below priorCDR_score
    windows_below_priorTransition_score = windows_mean_df[windows_mean_df['mean_value'] < priorTransition_score]
    # Create a BedTool object from the filtered windows
    windows_bedtool = pybedtools.BedTool.from_dataframe(windows_below_priorTransition_score)
    # Merge adjacent windows
    merged_windows = windows_bedtool.merge()

    # Create a BedTool object from CDR_df
    CDR_bedtool = pybedtools.BedTool.from_dataframe(CDR_df)
    
    # Perform bedtools intersect
    intersected_bedtool = merged_windows.intersect(CDR_bedtool, wa=True, wb=True)
    # Convert the result to a DataFrame and keep only columns from merged_windows
    intersected_df = intersected_bedtool.to_dataframe(names=[0, 1, 2, 'CDR_1', 'CDR_2', 'CDR_3', 'CDR_4'])
    intersected_df = intersected_df[[0, 1, 2]]
    
    # Convert intersected_df back to a BedTool object for subtraction
    intersected_bedtool = pybedtools.BedTool.from_dataframe(intersected_df)
    
    # Perform bedtools subtract
    priorTransition_bedtool = intersected_bedtool.subtract(CDR_bedtool)
    # Convert the result to a DataFrame
    priorTransition_df = priorTransition_bedtool.to_dataframe(names=[0, 1, 2])
    
    return priorTransition_df

In [163]:
def combine_and_save_bed(priorCDR_df, priorTransition_df, output_label, output_path):
    # Add the name column based on the origin
    priorCDR_df[3] = output_label
    priorTransition_df[3] = output_label + "_transition"
    
    # Concatenate the two DataFrames
    combined_df = pd.concat([priorCDR_df, priorTransition_df], ignore_index=True)
    
    # Sort the combined DataFrame by the second column (index 1)
    combined_df = combined_df.sort_values(by=1).reset_index(drop=True)
    
    # Save the combined DataFrame to a BED file
    combined_df.to_csv(output_path, sep='\t', header=False, index=False)

In [167]:
# Step 1: Filter bedMethyl file
filtered_bedMethyl = filter_bedMethyl(bedMethyl_path, mod_code)

print("filtered_bedMethyl:\n",filtered_bedMethyl)

# Step 2: Filter regions file
filtered_regions = filter_regions(cenSat_path, sat_type)

print("filtered_regions:\n",filtered_regions)

# Step 3: Intersect the files
intersected_df = intersect_files(filtered_bedMethyl, filtered_regions)

# Step 4: Create windows dataframe
windows_df = create_regions(intersected_df, window_size=window_size)

# Step 5: Calculate the mean of the 4th column within each window
windows_mean_df = calculate_mean_within_windows(intersected_df, windows_df)

print("windows_mean_df:\n",windows_mean_df)


filtered_bedMethyl:
                      0          1          2      10
0        chr10_MATERNAL       2498       2499    0.0
1        chr10_MATERNAL       2522       2523    0.0
2        chr10_MATERNAL       2528       2529    0.0
3        chr10_MATERNAL       2584       2585    0.0
4        chr10_MATERNAL       3102       3103  100.0
...                 ...        ...        ...    ...
1432418  chr10_MATERNAL  135910186  135910187  100.0
1432419  chr10_MATERNAL  135910272  135910273  100.0
1432420  chr10_MATERNAL  135910278  135910279  100.0
1432421  chr10_MATERNAL  135910296  135910297  100.0
1432422  chr10_MATERNAL  135910309  135910310  100.0

[1432423 rows x 4 columns]
filtered_regions:
                  0         1         2                               3    4  \
24  chr10_MATERNAL  39744030  40285609            active_hor(S1C10H1L)  100   
25  chr10_MATERNAL  40285609  40294936  mixedAlpha(S1C10H1-B,S1C10H1L)    0   
28  chr10_MATERNAL  40330733  42227817            active_ho

In [168]:
# Step 6: Calculate the priorCDR_score and priorTransition_score percentiles
priorCDR_score, priorTransition_score = calculate_percentiles(windows_mean_df, priorCDR_percent, priorTransition_percent)
print("scores: (CDR, Transition)")
print(priorCDR_score, priorTransition_score)

# Step 7: Create the priorCDR dataframe
priorCDR_df = create_priorCDR_dataframe(windows_mean_df, priorCDR_score, minCDR_size)
print('\npriorCDR_df:')
print(priorCDR_df)

# Step 8: Create the priorTransitions dataframe
priorTransitions_df = create_priorTransition_dataframe(windows_mean_df, priorTransition_score, priorCDR_df)
print('\npriorTransition_df:')
print(priorTransitions_df)

scores: (CDR, Transition)
18.1157000005 24.025633336

priorCDR_df:
                 0         1         2
30  chr10_MATERNAL  42096158  42102278
31  chr10_MATERNAL  42138998  42152258
33  chr10_MATERNAL  42210398  42278738

priorTransition_df:
                0         1         2
0  chr10_MATERNAL  42095138  42096158
1  chr10_MATERNAL  42152258  42154298


In [170]:
# Step 9: Combine and save DataFrame as a bedfile!
combine_and_save_bed(priorCDR_df, priorTransitions_df, output_label, "chr10_priorCDR_nb_test.bed")